# Create a MISP event from Microsoft Sentinel security incidents

## Introduction

- UUID: **00ba33ee-1402-4669-ad0b-fef256a36870**
- Started from [issue 34](https://github.com/MISP/misp-playbooks/issues/34)
- State: **Published** : demo version with **output**
- Purpose: This playbook extracts information from Microsoft Sentinel security incidents, parses the associated alerts and entities, and extracts useful indicators. A new MISP event is created with the incident summary, and the indicators are added to the MISP event. Sightings are also added to the indicators. At the end of the playbook, a summary is displayed and shared via Mattermost. The playbook uses credentials (tokens) obtained through an Azure App. Additionally, it includes a section on uploading custom logs to Sentinel, which was used during development and can be relevant for other purposes.
- Tags: [ "sentinel", "siem", "monitoring", "detection", "microsoft"]
- External resources: **Mattermost**, **Sentinel**
- Target audience: **CTI**, **CSIRT**, **SOC**

[![](https://mermaid.ink/img/pako:eNp9VMGOozAM_ZUoUi9V-QEOK43UPVSa7q6GuTUrTQoGog0JSsyuqop_35jSQBk6nBK_Z_vZDr7y3BbAU54kiTCoUEPKjofsF2u1vJyt_SPMAG02V2UUpuwqeKntv7yWDgUf7jU2-lWeQXsylFJ76HvWbzbCRCp7fROGhe-ocme9LTGDkM-APt0Pv4W5UXx3rpxsaxYp0YnNyERVJldFMBWAUml_Ooz3EZYaXMRe6OJHhMKgAn8Hv4_3CBdP5LIkYXtAyFFZw1yngaFTVQXOB-jbUtGqzIE41zYYFpJGIX-D-TRMZDjGJi3YJOsFg5Jzh8Aa2bbKVKOgQuUS7ahviHIPstDAsq5ppLtMRAetDXN-UsTX_BWFz_mzam-2Wc3sZplKn1jzkghaPCCKcQuUW-dASxpanDG5ze3zlCv8EqD4TP-ZHX68D9hqDnpHs3ST3jmZLRnLVA8xHjyJ7W9dXVf5mbFs3wO4eC0Ruzd2ml8jEcE11uMDdhL8Y688rY9IVyaukw_Bx9ZM_uRCvxczFlVJ6enXQsu222Mkbbfkyne8CXepirCzrhRIcKyhAdo8ghdQyk6HvSRMH6iyQ5tdTM5TdB3suLNdVfN02FA73rWFRNgrGd5KM1r7_6FdxPI?type=png)](https://mermaid.live/edit#pako:eNp9VMGOozAM_ZUoUi9V-QEOK43UPVSa7q6GuTUrTQoGog0JSsyuqop_35jSQBk6nBK_Z_vZDr7y3BbAU54kiTCoUEPKjofsF2u1vJyt_SPMAG02V2UUpuwqeKntv7yWDgUf7jU2-lWeQXsylFJ76HvWbzbCRCp7fROGhe-ocme9LTGDkM-APt0Pv4W5UXx3rpxsaxYp0YnNyERVJldFMBWAUml_Ooz3EZYaXMRe6OJHhMKgAn8Hv4_3CBdP5LIkYXtAyFFZw1yngaFTVQXOB-jbUtGqzIE41zYYFpJGIX-D-TRMZDjGJi3YJOsFg5Jzh8Aa2bbKVKOgQuUS7ahviHIPstDAsq5ppLtMRAetDXN-UsTX_BWFz_mzam-2Wc3sZplKn1jzkghaPCCKcQuUW-dASxpanDG5ze3zlCv8EqD4TP-ZHX68D9hqDnpHs3ST3jmZLRnLVA8xHjyJ7W9dXVf5mbFs3wO4eC0Ruzd2ml8jEcE11uMDdhL8Y688rY9IVyaukw_Bx9ZM_uRCvxczFlVJ6enXQsu222Mkbbfkyne8CXepirCzrhRIcKyhAdo8ghdQyk6HvSRMH6iyQ5tdTM5TdB3suLNdVfN02FA73rWFRNgrGd5KM1r7_6FdxPI)

# Playbook

- **Create a MISP event from Microsoft Sentinel security incidents**
    - Introduction
- **Preparation**
    - PR:1 Initialise environment
    - PR:2 Set helper variables
    - PR:3 Microsoft Sentinel
    - PR:4 MISP2Sentinel
    - PR:5 Azure App
    - PR:6 Microsoft Azure SDK
- **Sentinel**
    - SE:1 Sentinel incident details
    - SE:2 Sentinel alert and entity details
- **MISP**
    - MI:1 Create MISP event
    - MI:2 Add the MISP attributes
    - MI:3 Add MISP report
    - MI:4 Add sightings
- **Correlation**
    - CR:1 Correlation with MISP events
    - CR:2 Correlation with MISP feeds
- **Closure**
    - EN:1 Create the summary of the playbook 
    - EN:2 Print the summary
    - EN:3 Send a summary to Mattermost
    - EN:4 End of the playbook 
- **Extra section**
    - EX:1 Import logs into Sentinel
    - EX:2 Detection rules
- External references
- Technical details

# Preparation

## PR:1 Initialise environment

This section **initialises the playbook environment** and loads the required Python libraries. 

The credentials for MISP (**API key**) and other services are loaded from the file `keys.py` in the directory **vault**. A [PyMISP](https://github.com/MISP/PyMISP) object is created to interact with MISP and the active MISP server is displayed. By printing out the server name you know that it's possible to connect to MISP. In case of a problem PyMISP will indicate the error with `PyMISPError: Unable to connect to MISP`.

The contents of the `keys.py` file should contain at least :

```
misp_url="<MISP URL>"                  # The URL to our MISP server
misp_key="<MISP API KEY>"              # The MISP API key
misp_verifycert=<True or False>        # Ignore certificate errors
mattermost_playbook_user="<MATTERMOST USER>"
mattermost_hook="<MATTERMOST WEBHOOK>"

tenant_id = "<AZURE_TENANT>"      # Azure Tenant ID
client_id = "<CLIENT_ID>"         # Azure App client id
client_secret = "<CLIENT_SECRET>" # Azure App secret
workspace_id = "<WORKSPACE_ID>"   # Log Analytics workspace
```

In [2]:
# Initialise Python environment
import urllib3
import sys
import json
from pyfaup.faup import Faup
from prettytable import PrettyTable, MARKDOWN
from IPython.display import Image, display, display_markdown, HTML
from datetime import date
import requests
import uuid
from uuid import uuid4
from pymisp import *
from pymisp.tools import GenericObjectGenerator

import re
import time
from datetime import datetime

import copy

from azure.identity import ClientSecretCredential
import ast

# Load the credentials
sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("The \033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from the keys file.")

# Create the PyMISP object
misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp_headers = {"Authorization": misp_key,  "Content-Type": "application/json", "Accept": "application/json"}
print("I will use the MISP server \033[92m{}\033[90m for this playbook.".format(misp_url))

azure_credential = ClientSecretCredential(tenant_id, client_id, client_secret)
print("Created \033[92mAzure Credential\033[90m object.")

The Python libraries are loaded and the credentials are read from the keys file.
I will use the MISP server https://misp.demo.cudeso.be/ for this playbook.
Created Azure Credential object.


## PR:2 Set helper variables

This cell contains **helper variables** that are used in this playbook. Their usage is explained in the next steps of the playbook.

- `playbook_config` : the configuration of the playbook
- `playbook_results` : the results of the playbook

In [3]:
playbook_config = {
    "sentinel_entity_type_map": {
        "host": ["HostName", "hostname"],
        "ip": ["Address", "ip-dst"],
        "url": ["Url", "url"],
        "dns": ["DomainName", "hostname"],
        "filehash": ["Value", "sha256"],
    },
    "azure_token_scope": "https://api.loganalytics.io/.default",
    "azure_query_url": "https://api.loganalytics.io/v1/workspaces/",
    "correlation_published": True,
    "correlation_limit": 1000,
}

playbook_results = {
    "SecurityIncidents": {},
    "SecurityIncident_summary": {},
    "SecurityAlerts": [],
    "event": False,
    "indicators": {},
    "related_feeds": [],
    "related_events": [],
}


## PR:3 Microsoft Sentinel

Microsoft [Sentinel](https://learn.microsoft.com/en-us/azure/sentinel/overview?tabs=azure-portal) is a cloud-native security information and event management (SIEM) system. In Sentinel, **incidents** are collections of related alerts representing a potential security breach or threat. **Alerts** are individual notifications generated by detection rules or analytics that indicate possible security issues, such as suspicious activity or policy violations. **Entities** are the objects or elements involved in the alerts and incidents, such as users, IP addresses, hosts, files, or processes, providing context and details needed for investigation and response.

In this playbook, we use the entities associated with alerts and incidents to generate MISP events. The alerts can potentially be triggered by **threat intelligence** coming from your MISP server.

## PR:4 MISP2Sentinel

The MISP to Sentinel integration allows you to upload indicators from MISP to Microsoft Sentinel. It relies on PyMISP to get indicators from MISP and an **Azure App** with a Threat Intelligence Data Connector in Azure.

The integration is available on GitHub at [MISP2Sentinel](https://github.com/cudeso/misp2sentinel).

## PR:5 Azure App

For this playbook, you can either use the Azure App you're using for sending threat indicators from MISP to Microsoft Sentinel, or you can set up a new app with the proper permissions. The steps and requirements to achieve this are documented in the [installation](https://github.com/cudeso/misp2sentinel?tab=readme-ov-file#azure) section of MISP2Sentinel.

## PR:6 Microsoft Azure SDK

We do not use the Microsoft [Azure SDK](https://pypi.org/project/azure-mgmt-securityinsight/) for Python (such as `SecurityInsights`), simply because the SDK does not support querying the incident and alert *entities*. To work around this shortcoming, the playbook executes **KQL queries** to obtain the same information and then parses the returned results.

# Sentinel

This playbook section interacts with **Microsoft Sentinel**, via the credentials obtained from the Azure App.

## SE:1 Sentinel incident details

The first step in the playbook is to query the Sentinel incident details (**SecurityIncident**). Define the Sentinel incident ID (or IDs) in `sentinel_incidents`. You can supply one ID or multiple as a list. This is the **incident number** (internally stored in the field `IncidentNumber` in table `SecurityIncident`) displayed in the Azure interface and not the incident name, which is in the incident URL (and internally stored in `IncidentName`). 

This KQL is executed to obtain the incident information:
```kql
SecurityIncident 
| where IncidentNumber == {sentinel_incident_number}
| sort by TimeGenerated desc
| top 1 by TimeGenerated
| project Title, Severity, Status, Classification, CreatedTime, AlertIds, IncidentUrl, Type, Comments, AdditionalData
```

In [4]:
# Sentinel incident
sentinel_incidents = [266,267,268]

# Always make sure sentinel_incidents is a list
if not isinstance(sentinel_incidents, list):
    sentinel_incidents = [sentinel_incidents]
    
# Query URL
query_url = "{}{}/query".format(playbook_config["azure_query_url"], workspace_id)

# Get an Azure token
valid_azure_token = False
try:
    azure_token = azure_credential.get_token(playbook_config["azure_token_scope"]).token
    azure_headers = {
        "Authorization": f"Bearer {azure_token}",
        "Content-Type": "application/json"
    }
    print("Obtained a token.")
    valid_azure_token = True
except Exception as e:
    print("Unable to get a \033[91mtoken\033[90m", str(e))

if valid_azure_token:
    print("Query \033[92m{}\033[90m".format(query_url))

    for sentinel_incident_number in sentinel_incidents:
        print("Work with sentinel_incident_number: {}".format(sentinel_incident_number))
        # KQL
        query = f"""
        SecurityIncident 
        | where IncidentNumber == {sentinel_incident_number}
        | sort by TimeGenerated desc
        | top 1 by TimeGenerated
        | project Title, Severity, Status, Classification, CreatedTime, AlertIds, IncidentUrl, Type, Comments, AdditionalData
        """

        # Query Sentinel
        response = requests.post(query_url, headers=azure_headers, json={"query": query})
        if "tables" in response.json() and len(response.json()["tables"][0]["rows"]) == 1:
            row = response.json()["tables"][0]["rows"][0]

            # Extract comments
            comments = json.loads(row[8])
            extracted_comments = [
                {"message": comment["message"], "author": comment["author"]["name"]}
                for comment in comments
            ]

            # Extract tactics
            additional_data = ast.literal_eval(row[9])
            tactics = []
            if "tactics" in additional_data:
                tactics = additional_data["tactics"]

            # Add to SecurityIncident
            SecurityIncident = {
                "Title": row[0],
                "Severity": row[1],
                "Status": row[2],
                "Classification": row[3],
                "CreatedTime": row[4],
                "AlertIds": ast.literal_eval(row[5]),
                "IncidentUrl": row[6],
                "Type": row[7],
                "Comments": extracted_comments,
                "Tactics": tactics,
                "Number": sentinel_incident_number
            }
            playbook_results["SecurityIncidents"][sentinel_incident_number] = SecurityIncident
            print(" Found security incident \033[92m{}\033[90m".format(playbook_results["SecurityIncidents"][sentinel_incident_number]["Title"]))

            incident_summary = "### Sentinel incident {}\n".format(sentinel_incident_number)
            incident_summary += "- Title: **{}**\n".format(playbook_results["SecurityIncidents"][sentinel_incident_number]["Title"])
            incident_summary += "- Status: **{}**\n".format(playbook_results["SecurityIncidents"][sentinel_incident_number]["Status"])
            incident_summary += "- Severity: **{}**\n".format(playbook_results["SecurityIncidents"][sentinel_incident_number]["Severity"])
            if playbook_results["SecurityIncidents"][sentinel_incident_number]["Classification"].strip():
                incident_summary += "- Classification: **{}**\n".format(
                    playbook_results["SecurityIncidents"][sentinel_incident_number]["Classification"]
                )
            incident_summary += "- Type: **{}**\n".format(
                playbook_results["SecurityIncidents"][sentinel_incident_number]["Type"]
            )
            if playbook_results["SecurityIncidents"][sentinel_incident_number]["Tactics"]:
                incident_summary += "- Tactics: "
                for tactic in playbook_results["SecurityIncidents"][sentinel_incident_number]["Tactics"]:
                    incident_summary += "**{}** ".format(tactic)
                incident_summary += "\n"
            if playbook_results["SecurityIncidents"][sentinel_incident_number]["Comments"]:
                incident_summary += "- Comments:\n"
                for comment in playbook_results["SecurityIncidents"][sentinel_incident_number]["Comments"]:
                    incident_summary += "  - **{}** by {}\n".format(
                        comment["message"], comment["author"]
                    )
            incident_summary += "- Created: **{}**\n".format(
                playbook_results["SecurityIncidents"][sentinel_incident_number]["CreatedTime"]
            )
            incident_summary += "- Sentinel URL: {}\n".format(
                playbook_results["SecurityIncidents"][sentinel_incident_number]["IncidentUrl"]
            )
            incident_summary += "\n\n"
            playbook_results["SecurityIncidents"][sentinel_incident_number]["summary"] = incident_summary
        else:
            print("Unable to \033[91mextract incident information\033[90m from response")
else:
    print("Unable to continue without a valid \033[91mtoken\033[90m")


Obtained a token.
Query https://api.loganalytics.io/v1/workspaces/54456550-09a3-4547-b8e2-548f4e5d575c/query
Work with sentinel_incident_number: 266
 Found security incident TI alert cudeso
Work with sentinel_incident_number: 267
 Found security incident TI alert cudeso
Work with sentinel_incident_number: 268
 Found security incident TI alert cudeso


## SE:2 Sentinel alert and entity details

In this section, we query Sentinel to obtain details of all related alerts (**SecurityAlert**). The list of alert IDs was returned in the previous result. The playbook fetches the details of each alert, extracts the entities, and checks if any of these entities can be mapped to MISP attributes.

The KQL query to get the alert details is as follows:

```kql
SecurityAlert
| where SystemAlertId == '{alert_id}'
| project SystemAlertId, AlertName, Entities
```

This mapping of entities to MISP attributes is done using `playbook_config["sentinel_entity_type_map"]`. This dictionary contains the Sentinel entity types as keys. The corresponding values are lists with the first element being the key to retrieve the entity value, and the second element being the MISP attribute type. For example, `"host": ["HostName", "hostname"]`. Here, `host` is the Sentinel entity type, `HostName` is how the value is retrieved from the entity, and `hostname` is the MISP attribute type. The playbook includes a default set of MISP attributes, but you can also add your own.

In [5]:
print("Processing alert and entity details.")
print("Query \033[92m{}\033[90m".format(query_url))

for sentinel_incident_number in sentinel_incidents:
    alert_count = 0
    alert_summary = ""
    if playbook_results["SecurityIncidents"][sentinel_incident_number] and len(playbook_results["SecurityIncidents"][sentinel_incident_number]["AlertIds"]) > 0:
        playbook_results["SecurityIncidents"][sentinel_incident_number]["SecurityAlerts"] = []
        for alert_id in playbook_results["SecurityIncidents"][sentinel_incident_number]["AlertIds"]:
            query = f"""
                SecurityAlert
                | where SystemAlertId == '{alert_id}'
                | project SystemAlertId, AlertName, Entities
                """
            print(" Query for alert id \033[92m{}\033[90m".format(alert_id))
            response = requests.post(query_url, headers=azure_headers, json={"query": query})
            if "tables" in response.json() and len(response.json()["tables"][0]["rows"]) == 1:
                alert_count += 1
                row = response.json()["tables"][0]["rows"][0]
                alert = {"SystemAlertId": row[0], "AlertName": row[1], "Entities": row[2]}
                print(" Found alert \033[92m{}\033[90m".format(alert["AlertName"]))
                playbook_results["SecurityIncidents"][sentinel_incident_number]["SecurityAlerts"].append(alert)
                alert_summary += "- Alert: **{}**\n".format(alert["AlertName"])

                for entity in ast.literal_eval(alert["Entities"]):
                    entity_type = entity.get("Type")                
                    print("  Found entity type \033[92m{}\033[90m".format(entity_type))
                    if entity_type in playbook_config["sentinel_entity_type_map"]:
                        key = playbook_config["sentinel_entity_type_map"][entity_type][0]
                        value = entity.get(key)
                        if value not in playbook_results["indicators"]:
                            indicator = {"misp-attribute-type": playbook_config["sentinel_entity_type_map"][entity_type][1],
                                         "value": value,
                                         "alert": alert["AlertName"],
                                         "incident": sentinel_incident_number}
                            print("   Extracted indicator from {} {}".format(key, value))
                            playbook_results["indicators"][value] = indicator
                            alert_summary += "  - Entity: **{}**, transformed to **{}** {} \n".format(entity_type, value, indicator["misp-attribute-type"])
                        else:
                            print("   Skip value {} because it's already in indicator list".format(value))
                    else:
                        print("   Skip because {} - {} is not part of sentinel_entity_type_map".format(entity_type, entity))
                        alert_summary += "  - Entity: **{}** \n".format(entity_type)
        alert_summary = "#### Alerts\n Found **{}** alert(s) with below details.\n{}".format(alert_count, alert_summary)
        playbook_results["SecurityIncidents"][sentinel_incident_number]["alert_summary"] = alert_summary
        print(" Finished query")
    else:
        print("No security incident information obtained")
print("Finished")

Processing alert and entity details.
Query https://api.loganalytics.io/v1/workspaces/54456550-09a3-4547-b8e2-548f4e5d575c/query
 Query for alert id 680dee3d-9b30-11c3-c663-68cbd8881873
 Found alert TI alert cudeso
  Found entity type host
   Extracted indicator from HostName DC1
  Found entity type ip
   Extracted indicator from Address 185.195.237.123
  Found entity type filehash
   Extracted indicator from Value b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4
  Found entity type url
   Extracted indicator from Url https://message.ooguy.com
  Found entity type dns
   Extracted indicator from DomainName message.ooguy.com
 Finished query
 Query for alert id 0cf32c7a-256a-f66a-9a6e-82b02a365904
 Found alert TI alert cudeso
  Found entity type host
   Skip value DC1 because it's already in indicator list
  Found entity type ip
   Extracted indicator from Address 191.96.53.132
  Found entity type filehash
   Extracted indicator from Value f830c3771d35237b4a63b946d7a0d187f5

# MISP

This playbook section interacts with MISP.

## MI:1 Create MISP event

The next cell **creates the MISP event** and stores the reference (UUID) to the event in the variable `playbook_results["event"]`. Note that instead of creating a new event, you can also reference an existing MISP event (with its UUID, and in this case, also update the eventid with the Event ID). 

In [6]:
# Create the PyMISP object for an event
sentinel_incident_title = ', '.join(map(str, sentinel_incidents))
event_title = "Investigation Sentinel {}".format(sentinel_incident_title)
event = MISPEvent()
event.info = event_title
event.distribution = Distribution.your_organisation_only
event.threat_level_id = ThreatLevel.low
event.analysis = Analysis.ongoing
event.set_date(date.today())

# Create the MISP event on the server side
misp_event = misp.add_event(event, pythonify=True)
playbook_results["event"] = misp_event.uuid
playbook_results["eventid"] = misp_event.id

# Add default tags for the event
misp.tag(playbook_results["event"], "tlp:amber")
misp.tag(playbook_results["event"], "event-classification:event-class=\"incident\"")
misp.tag(playbook_results["event"], "workflow:state=\"incomplete\"", local=True)

print("Continue the playbook with the new MISP event ID \033[92m{}\033[90m with title \033[92m{}\033[90m and UUID \033[92m{}\033[90m.".format(misp_event.id, misp_event.info, playbook_results["event"]))

Continue the playbook with the new MISP event ID 3501 with title Investigation Sentinel 266, 267, 268 and UUID 682a0968-3512-44de-8c7b-cfe15049345a.


## MI:2 Add the MISP attributes

After creating the MISP event, the attributes extracted from the entities are added to MISP. By default, detection is enabled (`to_ids` is set to True), and a comment referencing the Sentinel incident is added. The playbook also includes a link to the Sentinel incident, as well as a comment attribute with the Sentinel incident ID. The latter can be useful for future **correlation** purposes.

In [7]:
print("Adding attributes to the MISP event.")
table = PrettyTable()
table.field_names = ["Source", "Value", "Type"]
table.align["Value"] = "l"
table.align["Type"] = "l"

safe_copy = copy.copy(playbook_results["indicators"]) # Avoid altering list while iterating over values
for value in safe_copy:
    attribute_comment = "Sentinel incident #{}".format(playbook_results["indicators"][value]["incident"])    
    attribute = MISPAttribute()
    attribute.type = playbook_results["indicators"][value]["misp-attribute-type"]
    attribute.value = value
    attribute.to_ids = True
    attribute.disable_correlation = False
    attribute.comment = attribute_comment
    result = misp.add_attribute(playbook_results["event"], attribute, pythonify=True)
    if "errors" not in result:
        print(" Added attribute {} of type {}".format(attribute.value, attribute.type))
        table.add_row(["Incident #{}".format(playbook_results["indicators"][value]["incident"]), value, attribute.type])
    else:
        print(" Failed to add attribute {} of type {} - {}".format(attribute.value, attribute.type, result))
        del(playbook_results["indicators"][value])
print("Finished adding attributes.")

print("Adding reference link to Sentinel incident")
for incident in playbook_results["SecurityIncidents"]:
    attribute = MISPAttribute()
    attribute.type = "link"
    attribute.value = playbook_results["SecurityIncidents"][incident]["IncidentUrl"]
    attribute.to_ids = False
    attribute.disable_correlation = False
    attribute.comment = "Sentinel incident #{} - {}".format(incident, playbook_results["SecurityIncidents"][incident]["Title"])
    result = misp.add_attribute(playbook_results["event"], attribute, pythonify=True)
    
    # Adding a comment with incident number to allow for correlation if we add additional events for the same incident
    attribute = MISPAttribute()
    attribute.type = "comment"
    attribute.value = "Sentinel incident #{} - {}".format(incident, playbook_results["SecurityIncidents"][incident]["Title"])
    attribute.to_ids = False
    attribute.disable_correlation = False
    result = misp.add_attribute(playbook_results["event"], attribute, pythonify=True)    
print("Finished adding reference link.")

print("\n")
print(table.get_string(sortby="Source"))
table_mispindicators = table

print("Finished creating table")

Adding attributes to the MISP event.


Something went wrong (403): {'saved': False, 'name': 'Could not add Attribute', 'message': 'Could not add Attribute', 'url': '/attributes/add', 'errors': {'value': ['Hostname has an invalid format. Please double check the value or select type "other".']}}


 Failed to add attribute DC1 of type hostname - {'errors': (403, {'saved': False, 'name': 'Could not add Attribute', 'message': 'Could not add Attribute', 'url': '/attributes/add', 'errors': {'value': ['Hostname has an invalid format. Please double check the value or select type "other".']}})}
 Added attribute 185.195.237.123 of type ip-dst
 Added attribute b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4 of type sha256
 Added attribute https://message.ooguy.com of type url
 Added attribute message.ooguy.com of type hostname
 Added attribute 191.96.53.132 of type ip-dst
 Added attribute f830c3771d35237b4a63b946d7a0d187f5aaa4240e965d74070b7d72b6fba210 of type sha256
 Added attribute https://dmsz.org of type url


Something went wrong (403): {'saved': False, 'name': 'Could not add Attribute', 'message': 'Could not add Attribute', 'url': '/attributes/add', 'errors': {'value': ['Hostname has an invalid format. Please double check the value or select type "other".']}}


 Added attribute dmsz.org of type hostname
 Failed to add attribute BACKUP1 of type hostname - {'errors': (403, {'saved': False, 'name': 'Could not add Attribute', 'message': 'Could not add Attribute', 'url': '/attributes/add', 'errors': {'value': ['Hostname has an invalid format. Please double check the value or select type "other".']}})}
 Added attribute 45.9.191.183 of type ip-dst
 Added attribute 776d427a19d8389464f855b2f70e0ac11e896162a9f9b50bcb23f0f0aea5044f of type sha256
 Added attribute https://cloud.keepasses.com of type url
 Added attribute cloud.keepasses.com of type hostname
Finished adding attributes.
Adding reference link to Sentinel incident
Finished adding reference link.


+---------------+------------------------------------------------------------------+----------+
|     Source    | Value                                                            | Type     |
+---------------+------------------------------------------------------------------+----------+
| Incident #

## MI:3 Add MISP report

The parsing of the incident and alert details created a Markdown summary. This summary is now added as a MISP report.

In [8]:
print("Creating MISP event reports.")

for incident in playbook_results["SecurityIncidents"]:
    summary_iv = ""
    summary_iv += playbook_results["SecurityIncidents"][incident]["summary"]
    summary_iv += "\n\n"
    summary_iv += playbook_results["SecurityIncidents"][incident]["alert_summary"]

    report_title = "Report for Sentinel incident #{} - {}".format(incident, playbook_results["SecurityIncidents"][incident]["Title"])
    print(" MISP report \033[92m{}\033[90m".format(report_title))
    chunk_size = 61500

    for i in range(0, len(summary_iv), chunk_size):
        chunk = summary_iv[i:i + chunk_size]
        event_report = MISPEventReport()
        event_title_edit = report_title
        if i > 0:
            event_title_edit = "{} ({} > {})".format(report_title, i, i + chunk_size)
        event_report.name = event_title_edit
        event_report.content = chunk
        result = misp.add_event_report(playbook_results["eventid"], event_report)
        if "EventReport" in result:
            print(" Report ID: \033[92m{}\033[90m".format(result.get("EventReport", {}).get("id", 0)))
        else:
            print(" Failed to create report for \033[91m{}\033[90m.".format(report_title))

print("Finished.\n")


Creating MISP event reports.
 MISP report Report for Sentinel incident #266 - TI alert cudeso
 Report ID: 917
 MISP report Report for Sentinel incident #267 - TI alert cudeso
 Report ID: 918
 MISP report Report for Sentinel incident #268 - TI alert cudeso
 Report ID: 919
Finished.



## MI:4 Add sightings

Now we'll add a MISP sighting for the entitites associated with the Sentinel incident. You can set the source with `sighting_source` and the sighting type with `sighting_type`. For the latter, these values exist
- 0 = True **sighting**, the most common used option
- 1 = **False positive** sighting
- 2 = **Expiration** sighting

The timestamp of the sighting corresponds with the time the playbook is executed. As an extension you could use the alert timestamp instead.

In [9]:
# Add sightings to MISP
sighting_source = "Set by playbook, detected in Sentinel incident"
sighting_type = 0  # Sighting types: 0=sighting ; 1=false positive ; 2=expiration

print("Adding sightings to MISP.")

for value in playbook_results["indicators"]:
    #dt = datetime.strptime(hit["@timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ")
    #dt = dt.replace(tzinfo=pytz.UTC)
    sighting_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sighting_source_add = "{} #{}".format(sighting_source, playbook_results["indicators"][value]["incident"])
    data = {"value": value, "type": sighting_type, "source": sighting_source_add, "date_sighting": sighting_timestamp}
    request = misp._prepare_request("POST", "{}/sightings/add".format(misp_url), data=data)
    if "Sighting" in request.json():
        print(" Adding \033[92m{}\033[90m at {}".format(value, sighting_source_add))
    else:
        print(" Unable to add sighting \033[91m{}\033[90m {}".format(value, request.text))
print("Finished adding sightings.")

Adding sightings to MISP.
 Adding 185.195.237.123 at Set by playbook, detected in Sentinel incident #266
 Adding b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4 at Set by playbook, detected in Sentinel incident #266
 Adding https://message.ooguy.com at Set by playbook, detected in Sentinel incident #266
 Adding message.ooguy.com at Set by playbook, detected in Sentinel incident #266
 Adding 191.96.53.132 at Set by playbook, detected in Sentinel incident #267
 Adding f830c3771d35237b4a63b946d7a0d187f5aaa4240e965d74070b7d72b6fba210 at Set by playbook, detected in Sentinel incident #267
 Adding https://dmsz.org at Set by playbook, detected in Sentinel incident #267
 Adding dmsz.org at Set by playbook, detected in Sentinel incident #267
 Adding 45.9.191.183 at Set by playbook, detected in Sentinel incident #268
 Adding 776d427a19d8389464f855b2f70e0ac11e896162a9f9b50bcb23f0f0aea5044f at Set by playbook, detected in Sentinel incident #268
 Adding https://cloud.keepasses.com 

# Correlation

## CR:1 Correlation with MISP events

When the event and attributes are added to MISP it will immediately show the related events and OSINT feed matches in the web interface. We also want that information to be included in the playbook results and summary. 

Only published events (`correlation_published`) are take into account. There is a default limit of 1000 hits (`correlation_limit`).

In [10]:
# Code block to query MISP and find the correlations
for value in playbook_results["indicators"]:
    search_match = misp.search("events", value=value, published=playbook_config["correlation_published"],
                                        limit=playbook_config["correlation_limit"], pythonify=True)
    if len(search_match) > 0:
        for event in search_match:
            if event.uuid != playbook_results["event"]:   # Skip the event we just created for this playbook
                print("Found match for {} in \033[92m{}\033[90m in \033[92m{}\033[90m".format(value, event.id, event.info))
                entry = {"source": "MISP", "org": event.org.name, "event_id": event.id, "event_info": event.info,
                                     "date": event.date, "value": value, "incident": playbook_results["indicators"][value]["incident"]}
                playbook_results["related_events"].append(entry)
    else:
        print("\033[93mNo correlating MISP events\033[90m found for {}.".format(value))
print("Finished correlating with MISP events.\n\n")

Found match for 185.195.237.123 in 3493 in Operation Crimson Palace: Sophos threat hunting unveils multiple clusters of Chinese state-sponsored activity targeting Southeast Asian government
Found match for b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4 in 3493 in Operation Crimson Palace: Sophos threat hunting unveils multiple clusters of Chinese state-sponsored activity targeting Southeast Asian government
No correlating MISP events found for https://message.ooguy.com.
Found match for message.ooguy.com in 3493 in Operation Crimson Palace: Sophos threat hunting unveils multiple clusters of Chinese state-sponsored activity targeting Southeast Asian government
Found match for 191.96.53.132 in 3493 in Operation Crimson Palace: Sophos threat hunting unveils multiple clusters of Chinese state-sponsored activity targeting Southeast Asian government
Found match for f830c3771d35237b4a63b946d7a0d187f5aaa4240e965d74070b7d72b6fba210 in 3493 in Operation Crimson Palace: Sophos th

### MISP events correlation table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost.

In [11]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Event", "Event ID", "Incident"]
table.align["Value"] = "l"
table.align["Event"] = "l"
table.align["Event ID"] = "l"
table._max_width = {"Event": 50}
for match in playbook_results["related_events"]:
    table.add_row([match["source"], match["value"], match["event_info"], match["event_id"], match["incident"]])
print(table.get_string(sortby="Value"))
table_mispevents = table

+--------+------------------------------------------------------------------+----------------------------------------------------+----------+----------+
| Source | Value                                                            | Event                                              | Event ID | Incident |
+--------+------------------------------------------------------------------+----------------------------------------------------+----------+----------+
|  MISP  | 185.195.237.123                                                  | Operation Crimson Palace: Sophos threat hunting    | 3493     |   266    |
|        |                                                                  | unveils multiple clusters of Chinese state-        |          |          |
|        |                                                                  | sponsored activity targeting Southeast Asian       |          |          |
|        |                                                                  | gove

## CR:2 Correlation with MISP feeds

Search the MISP feeds for events that match with one of the fingerprints you specified earlier. The results highly depend on the feeds you have enabled. 

In [12]:
print("Search in MISP feeds.")
misp_cache_url = "{}/feeds/searchCaches/".format(misp_url)
match = False
for value in playbook_results["indicators"]:
    # Instead of GET, use POST (https://github.com/MISP/MISP/issues/7478)
    cache_results = requests.post(misp_cache_url, headers=misp_headers, verify=misp_verifycert, json={"value": value})
    for result in cache_results.json():
        if "Feed" in result:
            match = True
            print(" Found \033[92m{}\033[90m in \033[92m{}\033[90m.".format(value, result["Feed"]["name"]))
            for match in result["Feed"]["direct_urls"]:
                entry = {"source": "Feeds", "feed_name": result["Feed"]["name"], "match_url": match["url"], "value": value, "incident": playbook_results["indicators"][value]["incident"]}
                playbook_results["related_feeds"].append(entry)

print("Finished searching in MISP feeds.")
if not match:
    print("\033[93mNo correlating information found in MISP feeds.")

Search in MISP feeds.
 Found 185.195.237.123 in The Botvrij.eu Data.
 Found b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4 in The Botvrij.eu Data.
 Found message.ooguy.com in The Botvrij.eu Data.
 Found 191.96.53.132 in The Botvrij.eu Data.
 Found f830c3771d35237b4a63b946d7a0d187f5aaa4240e965d74070b7d72b6fba210 in The Botvrij.eu Data.
 Found dmsz.org in The Botvrij.eu Data.
 Found 45.9.191.183 in The Botvrij.eu Data.
 Found 776d427a19d8389464f855b2f70e0ac11e896162a9f9b50bcb23f0f0aea5044f in The Botvrij.eu Data.
 Found cloud.keepasses.com in The Botvrij.eu Data.
Finished searching in MISP feeds.


### MISP feed correlations table

The correlation results are now stored in `playbook_results`. Execute the next cell to display them in a table format. The table is also included in the summary for Mattermost.

In [13]:
# Put the correlations in a pretty table. We can use this table later also for the summary
table = PrettyTable()
table.field_names = ["Source", "Value", "Feed", "Feed URL", "Incident"]
table.align["Value"] = "l"
table.align["Feed"] = "l"
table.align["Feed URL"] = "l"
table._max_width = {"Event": 50}
for match in playbook_results["related_feeds"]:
    table.add_row([match["source"], match["value"], match["feed_name"], match["match_url"], match["incident"]])
print(table.get_string(sortby="Value"))
table_mispfeeds = table

+--------+------------------------------------------------------------------+---------------------+---------------------------------------------------------------------------------------+----------+
| Source | Value                                                            | Feed                | Feed URL                                                                              | Incident |
+--------+------------------------------------------------------------------+---------------------+---------------------------------------------------------------------------------------+----------+
| Feeds  | 185.195.237.123                                                  | The Botvrij.eu Data | https://misp.demo.cudeso.be/feeds/previewEvent/2/f48f7c30-fe6f-4854-b27e-f86a308da714 |   266    |
| Feeds  | 191.96.53.132                                                    | The Botvrij.eu Data | https://misp.demo.cudeso.be/feeds/previewEvent/2/f48f7c30-fe6f-4854-b27e-f86a308da714 |   267    |
| Fee

# Closure

In this **closure** or end step we create a **summary** of the actions that were performed by the playbook. The summary is printed and can also be send to a chat channel. 

## EN:1 Create the summary of the playbook 

The next section creates a summary and stores the output in the variable `summary` in Markdown format. It also stores an intro text in the variable `intro`. These variables can later be used when sending information to Mattermost or TheHive.

In [14]:
summary = "# MISP Playbook summary\nCreate a MISP event from Microsoft Sentinel security incidents \n\n"

current_date = datetime.now()
formatted_date = current_date.strftime("%Y-%m-%d")
summary += "## Overview\n\n"
summary += "This concerned the investigation of security incident **{}**\n\n".format(event.info)
summary += "- Date: **{}**\n".format(formatted_date)
summary += "- Event: **{}** - UUID: {}\n".format(event.id, event.uuid)
summary += "\n\n"

summary += "## Incidents\n"
for incident in playbook_results["SecurityIncidents"]:
    summary += playbook_results["SecurityIncidents"][incident]["summary"]
    summary += playbook_results["SecurityIncidents"][incident]["alert_summary"]
    summary += "\n\n"
    
summary += "## MISP correlations\n\n"
summary += "### Events\n\n"
table_mispevents.set_style(MARKDOWN)
summary += table_mispevents.get_string(sortby="Value")
summary += "\n\n"

summary += "### OSINT feeds\n\n"
table_mispfeeds.set_style(MARKDOWN)
summary += table_mispfeeds.get_string(sortby="Value")
summary += "\n\n"

summary += "## MISP indicators\n\n"
table_mispindicators.set_style(MARKDOWN)
summary += table_mispindicators.get_string(sortby="Type")
summary += "\n\n"


print("The \033[92msummary\033[90m of the playbook is available.\n")

The summary of the playbook is available.



## EN:2 Print the summary

In [ ]:
print(summary)
# Or print with parsed markdown
#display_markdown(summary, raw=True)

## EN:3 Send a summary to Mattermost

Now you can send the summary to Mattermost. You can send the summary in two ways by selecting one of the options for the variable `send_to_mattermost_option` in the next cell.

- The default option where the entire summary is in the **chat**, or
- a short intro and the summary in a **card**

For this playbook we rely on a webhook in Mattermost. You can add a webhook by choosing the gear icon in Mattermost, then choose Integrations and then **Incoming Webhooks**. Set a channel for the webhook and lock the webhook to this channel with *"Lock to this channel"*.

In [16]:
send_to_mattermost_option = "via a chat message"
#send_to_mattermost_option = "via a chat message with card"

In [17]:
message = False
if send_to_mattermost_option == "via a chat message":
    message = {"username": mattermost_playbook_user, "text": summary}
elif send_to_mattermost_option == "via a chat message with card":
    message = {"username": mattermost_playbook_user, "text": intro, "props": {"card": summary}}

if message:
    r = requests.post(mattermost_hook, data=json.dumps(message))
    r.raise_for_status()
if message and r.status_code == 200:
    print("Summary is \033[92msent to Mattermost.\n")
else:
    print("\033[91mFailed to sent summary\033[90m to Mattermost.\n")

Summary is sent to Mattermost.



## EN:4 End of the playbook 

In [18]:
print("\033[92m End of the playbook")


 End of the playbook


# Extra section

## EX:1 Import logs into Sentinel

This section demonstrates how to import custom logs into a table in Sentinel. It uses a simple CSV file (defined in `csv_data`) and uploads it to your Log Analytics workspace.

To perform the import, you need to specify the following:

- `azure_customer_id`: The **Log Analytics Workspace ID**.
- `azure_shared_key`: The **Log Analytics Primary Key** (found under Log Analytics > Settings > Agents > Log Analytics agent instructions).
- `azure_log_type`: The name of the **custom log table**.

In [ ]:
# Required libraries
import csv
import io
import base64
import requests
import hmac
import hashlib

# Build the signature
def build_signature(customer_id, shared_key, date, content_length, method, content_type, resource):
    x_headers = 'x-ms-date:' + date
    string_to_hash = method + '\n' + str(content_length) + '\n' + content_type + '\n' + x_headers + '\n' + resource
    bytes_to_hash = bytes(string_to_hash, 'utf-8')
    decoded_key = base64.b64decode(shared_key)
    encoded_hash = base64.b64encode(hmac.new(decoded_key, bytes_to_hash, hashlib.sha256).digest()).decode('utf-8')
    authorization = "SharedKey {}:{}".format(customer_id, encoded_hash)
    return authorization


# Post data
def post_data(customer_id, shared_key, body, log_type):
    method = 'POST'
    content_type = 'application/json'
    resource = '/api/logs'
    rfc1123date = datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')
    content_length = len(body)
    signature = build_signature(customer_id, shared_key, rfc1123date, content_length, method, content_type, resource)
    uri = 'https://' + customer_id + '.ods.opinsights.azure.com' + resource + '?api-version=2016-04-01'

    headers = {
        'Content-Type': content_type,
        'Authorization': signature,
        'Log-Type': log_type,
        'x-ms-date': rfc1123date,
        'time-generated-field': 'TimeGenerated'
    }

    response = requests.post(uri, data=body, headers=headers)
    if response.status_code >= 200 and response.status_code <= 299:
        print(response, response.status_code, vars(response))
        print('Accepted')
    else:
        print('Error: {}'.format(response.status_code))
        print(response.text)

        
# Function to convert CSV string to JSON
def csv_string_to_json(csv_string):
    csv_reader = csv.DictReader(io.StringIO(csv_string))
    json_array = [row for row in csv_reader]
    return json.dumps(json_array, indent=4)

In [ ]:
# Adjust with your credentials
azure_customer_id = ''  # Replace with your Log Analytics Workspace ID
azure_shared_key = ''  # Replace with your Log Analytics Primary Key
azure_log_type = 'CudesoDemoSyslog' # Log name

# CSV data as a string
csv_data = """Computer,HostIP,HostName,DestinationIP,FileHash,ProcessName,DestinationDomain,DestinationURL
DC1,192.168.1.1,DC1.demo.cudeso.be,185.195.237.123,b708dd11942c3e87a8987bdf83f7ea603425ae75fc25a306f54f1087df4198b4,curl.exe,message.ooguy.com,https://message.ooguy.com
DC1,192.168.1.1,DC1.demo.cudeso.be,191.96.53.132,f830c3771d35237b4a63b946d7a0d187f5aaa4240e965d74070b7d72b6fba210,curl.exe,dmsz.org,https://dmsz.org
BACKUP1,192.168.1.20,BACKUP1.demo.cudeso.be,45.9.191.183,776d427a19d8389464f855b2f70e0ac11e896162a9f9b50bcb23f0f0aea5044f,certutil.exe,cloud.keepasses.com,https://cloud.keepasses.com"""

# Post data
print("Posting data")
post_data(azure_customer_id, azure_shared_key, csv_string_to_json(csv_data), azure_log_type)
print("Finished posting data")

## EX:2 Detection rules

After uploading the CSV files, you can add **detection rules**. In this demo, the detection rules trigger on entries from the CSV files that match previously uploaded threat intelligence in Sentinel. To set up these rules, navigate to your Log Analytics workspace, select **Configuration**, and then **Analytics**. Here, you can create a new rule to trigger based on the results of a specific query. For this example, use the following KQL query, which triggers on network activity:

```kql
let indicator_list = (
    ThreatIntelligenceIndicator
    | where ExpirationDateTime > now()
    | where ExpirationDateTime < datetime(9999-12-31)
    | where TimeGenerated > ago(1d)
    | where Active == true
    | where isnotempty(NetworkDestinationIP) or isnotempty(NetworkSourceIP)
    | extend NetworkIPs = case(
        isnotempty(NetworkDestinationIP) and isnotempty(NetworkSourceIP), NetworkDestinationIP,
        isnotempty(NetworkDestinationIP), NetworkDestinationIP,
        isnotempty(NetworkSourceIP), NetworkSourceIP,
        ""
    )
    | distinct NetworkIPs 
    | project NetworkIPs 
);
CudesoDemoSyslog_CL
| where DestinationIP_s has_any (indicator_list)
```

In the detection rule, under **Set rule logic**, you can map the **Entities** to specific fields. For example, map FileHash to FileHash_s, URL to DestinationURL_s, etc.

## External references

- [The MISP Project](https://www.misp-project.org/)
- [Mattermost](https://mattermost.com/)
- [MISP2Sentinel](https://github.com/cudeso/misp2sentinel)

## Technical details 

### Documentation

This playbook requires these Python **libraries** to exist in the environment where the playbook is executed. You can install them with `pip install <library>`.

```
PrettyTable
ipywidgets
azure-identity
```

### Colour codes

The output from Python displays some text in different colours. These are the colour codes

```
Red = '\033[91m'
Green = '\033[92m'
Blue = '\033[94m'
Cyan = '\033[96m'
White = '\033[97m'
Yellow = '\033[93m'
Magenta = '\033[95m'
Grey = '\033[90m'
Black = '\033[90m'
Default = '\033[99m'
```